# Timeseries classification with a Transformer model


In [17]:
import pandas as pd
from pandas import read_csv
from tensorflow.keras.utils import to_categorical

In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

path = 'gdrive/My Drive/Esame bellotti/Consegna Github/dataset/'

data = ["S08R01"]

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [14]:
def split_dataframe(df):
  #Split in train e test
  n_row = df.shape[0]
  up = round(n_row * 0.7)
  df_train = df.iloc[:up,:]
  df_test = df.iloc[up+1:,:]
  return df_train, df_test

In [13]:
def balance_dataset(df):
  """
  Calcolo quanti elementi sono da togliere
  Tolgo random un elemento da x e il corrispondente y
  """
  print("Bilanciamento")
  print(df['N'].value_counts())
  FOG = df[df.N == 2].shape[0]
  NOT_FOG = df[df.N == 1].shape[0]
  
  #print("Fog casi: " + str(FOG) + " NON FOG: " + str(NOT_FOG))
  if NOT_FOG > FOG:
    da_togliere = NOT_FOG - FOG

    new_df = df.drop(df[df['N'].eq(1)].sample(da_togliere).index)


  FOG = new_df[new_df.N == 2].shape[0]
  NOT_FOG = new_df[new_df.N == 1].shape[0]
  

  return new_df

In [12]:
def load_dataset(files):
    
    """
    This function reads the accelerometer data from a file
    Args:
        file_path: URL pointing to the CSV file
    Returns:
        A pandas dataframe
    """
    df = pd.DataFrame()
    for txt in files:
        read_df = pd.read_csv(path+txt+'.txt')
        s = read_df["N"] != 0
        tmp = read_df.loc[s]
        df = df.append(tmp)

    #df = segmentation(df)
    df = balance_dataset(df)

    df_train, df_test = split_dataframe(df)

    
    #trainX = np.asarray(df_train["Data"].values.tolist())
    trainX = np.asarray(df_train[["A0", "A1", "A2", "U0", "U1", "U2", "T1", "T2", "T3"]])
    trainy = np.asarray(df_train["N"])

    #testX = np.asarray(df_test["Data"].values.tolist())
    testX = np.asarray(df_test[["A0", "A1", "A2", "U0", "U1", "U2", "T1", "T2", "T3"]])
    testy = np.asarray(df_test["N"])


    #trainy = to_categorical(trainy) 
    #testy = to_categorical(testy)
    #print(trainX.shape, trainy.shape, testX.shape, testy.shape)

    return trainX, trainy, testX, testy

In [ ]:
import numpy as np


# def readucr(filename):
#     data = np.loadtxt(filename, delimiter="\t")
#     y = data[:, 0]
#     x = data[:, 1:]
#     return x, y.astype(int)


# root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

# x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
# x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

# print(type(x_train))

x_train, y_train, x_test, y_test = load_dataset(data)


x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

n_classes = len(np.unique(y_train))

idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

y_train[y_train == 1] = 0
y_test[y_test == 1] = 0

y_train[y_train == 2] = 1
y_test[y_test == 2] = 1

Bilanciamento
1    36425
2    12859
Name: N, dtype: int64


In [8]:
from tensorflow import keras
from tensorflow.keras import layers

We include residual connections, layer normalization, and dropout.
The resulting layer can be stacked multiple times.

The projection layers are implemented through `keras.layers.Conv1D`.

In [ ]:

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res


The main part of our model is now complete. We can stack multiple of those
`transformer_encoder` blocks and we can also proceed to add the final
Multi-Layer Perceptron classification head. Apart from a stack of `Dense`
layers, we need to reduce the output tensor of the `TransformerEncoder` part of
our model down to a vector of features for each data point in the current
batch. A common way to achieve this is to use a pooling layer. For
this example, a `GlobalAveragePooling1D` layer is sufficient.

In [ ]:

def build_model(input_shape,head_size,num_heads,ff_dim,num_transformer_blocks,mlp_units,dropout=0, mlp_dropout=0,):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)


## Train and evaluate

In [ ]:
input_shape = x_train.shape[1:]

print(input_shape)
print(x_train.shape)

print(x_train)


model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

hystory = model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)

In [ ]:
model.save('gdrive/My Drive/Esame bellotti/Modelli/Modello_paziente_8_bialanciato')



INFO:tensorflow:Assets written to: gdrive/My Drive/Esame bellotti/Modelli/Modello_paziente_8_bialanciato/assets


INFO:tensorflow:Assets written to: gdrive/My Drive/Esame bellotti/Modelli/Modello_paziente_8_bialanciato/assets


In [5]:
import numpy as np
from tensorflow.math import confusion_matrix
from matplotlib import pyplot as plt
import seaborn as sns

In [31]:


def show_confusion_matrix(validations, predictions):

    matrix = confusion_matrix(validations, predictions)
    plt.figure(figsize=(6, 4))
    sns.heatmap(matrix,
                cmap="coolwarm",
                linecolor='white',
                linewidths=1,
                xticklabels=['Not FOG', 'FOG'],
                yticklabels=['Not FOG', 'FOG'],
                annot=True,
                fmt="d")
    plt.title("Confusion Matrix")
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.show()

    tn = matrix[0][0]
    tp = matrix[1][1]
    fp = matrix[0][1]
    fn = matrix[1][0]
    print(matrix)


    spec = tn/(tn+fp)
    sens = tp/(tp+fn)
    prec = tp/(tp+fp)

    return spec, sens, prec

In [10]:
#Loading del modello 
model1 = keras.models.load_model('gdrive/My Drive/Esame bellotti/Consegna Github/Modelli transformer/Modello_paziente_1_bialanciato')

In [19]:

x_train, y_train, x_test, y_test = load_dataset(["S08R01"])
import time
start = time.time()
y_pred_test = model1.predict(x_test)
end = time.time()
print("Tempo di predizione")
print(start - end)



Temppo di predizione
-12.496735572814941


In [20]:
print(y_pred_test)

[[0.74814105 0.25185898]
 [0.70242685 0.2975732 ]
 [0.73648    0.26352   ]
 ...
 [0.36338782 0.6366122 ]
 [0.33993417 0.6600658 ]
 [0.3720118  0.6279882 ]]


In [ ]:
max_y_pred_test = np.argmax(y_pred_test, axis=1)
spec, sens, prec = show_confusion_matrix(y_test, max_y_pred_test)

print(np.min(max_y_pred_test))